In [1]:
import os
import json
import numpy as np
import pandas as pd
import re

from googleapiclient import discovery
from apiclient.discovery import build # libraries needed for google sheets API
from httplib2 import Http
from oauth2client import file, client, tools

import urllib
import requests

In [2]:
# reference using google API and Medium posts on how to access the Google API
# https://towardsdatascience.com/how-to-access-google-sheet-data-using-the-python-api-and-convert-to-pandas-dataframe-5ec020564f0e
# https://developers.google.com/sheets/api/guides/concepts#spreadsheet_id

# the following function was modified from Medium link sourced above and the quickstart.py file from google
def get_google_sheet(spreadsheet_id, range_name):
    """ Retrieve sheet data using OAuth credentials and Google Python API. """
    scopes = 'https://www.googleapis.com/auth/spreadsheets.readonly'
    # Setup the Sheets API
    store = file.Storage(os.getenv("HOME")+'/keys/'+'token.json')
    creds = store.get()
    if not creds or creds.invalid:
        flow = client.flow_from_clientsecrets(os.getenv("HOME")+'/keys/'+'credentials.json', scopes)
        creds = tools.run_flow(flow, store)
    service = build('sheets', 'v4', http=creds.authorize(Http()))

    # Call the Sheets API
    gsheet = service.spreadsheets().values().get(spreadsheetId=spreadsheet_id, range=range_name).execute()
    return gsheet

In [3]:
# this is a google sheet that was downloaded from https://data.medicare.gov/Hospital-Compare/Hospital-General-Information/xubh-q36u
# we had to manually look up the link to each mastercharge file on each hospitals' website
# google sheet link https://docs.google.com/spreadsheets/d/1F8yPe-2uMcAOOzRmYnFenC77GiXnJ0afnRXtBUYe5TQ/edit?usp=sharing

SHEET_ID = '1F8yPe-2uMcAOOzRmYnFenC77GiXnJ0afnRXtBUYe5TQ' 
RANGE = 'NYChospitals' # this is the range

gsheet = get_google_sheet(SHEET_ID, RANGE)

In [4]:
gsheet.keys() # check the keys of the gsheet

dict_keys(['range', 'majorDimension', 'values'])

In [5]:
## turn the list of hospitals from gsheet into pd df
nycHospitals = pd.DataFrame.from_records(gsheet['values'][1:], columns=gsheet['values'][0]) 

In [6]:
nycHospitals.head(3) # to see the dataframe

,in_nyc,Provider ID,Hospital Name,Main webpage link,DRG webpage link,All hospital charges link,Remarks on chargemaster link,format,Address,City,...,Readmission national comparison footnote,Patient experience national comparison,Patient experience national comparison footnote,Effectiveness of care national comparison,Effectiveness of care national comparison footnote,Timeliness of care national comparison,Timeliness of care national comparison footnote,Efficient use of medical imaging national comparison,Efficient use of medical imaging national comparison footnote,Location
0,TRUE,330204,BELLEVUE HOSPITAL CENTER,https://www.bellevuehospital.com/patient-pricing,NA,https://www.bellevuehospital.com/sites/default...,"csv file, no DRG",csv,462 FIRST AVENUE,NEW YORK,...,,Below the national average,,Same as the national average,,Below the national average,,Same as the national average,,"462 FIRST AVENUE NEW YORK, NY (40.740079, -73...."
1,TRUE,330009,BRONX-LEBANON HOSPITAL CENTER,https://www.bronxcare.org/about-us/paying-for-...,https://www.bronxcare.org/fileadmin/SiteFiles/...,https://www.bronxcare.org/fileadmin/SiteFiles/...,"2017 data, Excel file; for DRG, split into \nA...",xls,1276 FULTON AVENUE,BRONX,...,,Below the national average,,Same as the national average,,Below the national average,,Same as the national average,,"1276 FULTON AVENUE BRONX, NY (40.83175, -73.90..."
2,TRUE,330233,BROOKDALE HOSPITAL MEDICAL CENTER,http://www.brookdalehospital.org/charge-master...,http://www.brookdalehospital.org/assets/charge...,http://www.brookdalehospital.org/assets/brookd...,Excel file,xlsx,1 BROOKDALE PLAZA,BROOKLYN,...,,Below the national average,,Below the national average,,Below the national average,,Same as the national average,,"1 BROOKDALE PLAZA BROOKLYN, NY (40.6545, -73.9..."


In [7]:
nycHospitals['url'] = np.where(nycHospitals['DRG webpage link']=='NA',
                               nycHospitals['All hospital charges link'], 
                               nycHospitals['DRG webpage link'])

In [8]:
nycHospitals.columns

Index(['in_nyc', 'Provider ID', 'Hospital Name', 'Main webpage link',
       'DRG webpage link', 'All hospital charges link',
       'Remarks on chargemaster link', 'format', 'Address', 'City', 'State',
       'ZIP Code', 'County Name', 'Phone Number', 'Hospital Type',
       'Hospital Ownership', 'Emergency Services',
       'Meets criteria for meaningful use of EHRs', 'Hospital overall rating',
       'Hospital overall rating footnote', 'Mortality national comparison',
       'Mortality national comparison footnote',
       'Safety of care national comparison',
       'Safety of care national comparison footnote',
       'Readmission national comparison',
       'Readmission national comparison footnote',
       'Patient experience national comparison',
       'Patient experience national comparison footnote',
       'Effectiveness of care national comparison',
       'Effectiveness of care national comparison footnote',
       'Timeliness of care national comparison',
       'Timeli

In [9]:
nycHospitals.drop(['in_nyc', 'Main webpage link',
       'DRG webpage link', 'All hospital charges link',
       'Remarks on chargemaster link', 'Address', 'City', 'State', 'ZIP Code',
       'County Name', 'Phone Number', 'Hospital Type', 'Hospital Ownership',
       'Emergency Services', 'Meets criteria for meaningful use of EHRs',
       'Hospital overall rating', 'Hospital overall rating footnote',
       'Mortality national comparison',
       'Mortality national comparison footnote',
       'Safety of care national comparison',
       'Safety of care national comparison footnote',
       'Readmission national comparison',
       'Readmission national comparison footnote',
       'Patient experience national comparison',
       'Patient experience national comparison footnote',
       'Effectiveness of care national comparison',
       'Effectiveness of care national comparison footnote',
       'Timeliness of care national comparison',
       'Timeliness of care national comparison footnote',
       'Efficient use of medical imaging national comparison',
       'Efficient use of medical imaging national comparison footnote',
       'Location'], axis = 1, inplace = True)

In [10]:
nycHospitals.columns = ['providerId', 'hospitalName', 'format','url']

In [11]:
nycHospitals['tempName'] = nycHospitals['hospitalName'].str.translate ({ord(c): " " for c in "!@#$%^&*()[]{};:,./<>?\|`~-=_+"}).str.title().str.split()

In [12]:
nycHospitals

,providerId,hospitalName,format,url,tempName
0,330204,BELLEVUE HOSPITAL CENTER,csv,https://www.bellevuehospital.com/sites/default...,"[Bellevue, Hospital, Center]"
1,330009,BRONX-LEBANON HOSPITAL CENTER,xls,https://www.bronxcare.org/fileadmin/SiteFiles/...,"[Bronx, Lebanon, Hospital, Center]"
2,330233,BROOKDALE HOSPITAL MEDICAL CENTER,xlsx,http://www.brookdalehospital.org/assets/charge...,"[Brookdale, Hospital, Medical, Center]"
3,330056,BROOKLYN HOSPITAL CENTER AT DOWNTOWN CAMPUS,csv,https://www.tbh.org/sites/default/files/Brookl...,"[Brooklyn, Hospital, Center, At, Downtown, Cam..."
4,330196,CONEY ISLAND HOSPITAL,xlsx,https://hhinternet.blob.core.windows.net/uploa...,"[Coney, Island, Hospital]"
5,330128,ELMHURST HOSPITAL CENTER,xlsx,https://hhinternet.blob.core.windows.net/uploa...,"[Elmhurst, Hospital, Center]"
6,330193,FLUSHING HOSPITAL MEDICAL CENTER,xlsx,https://www.flushinghospital.org/sites/default...,"[Flushing, Hospital, Medical, Center]"
7,330240,HARLEM HOSPITAL CENTER,xlsx,https://hhinternetauto.blob.core.windows.net/u...,"[Harlem, Hospital, Center]"
8,330270,HOSPITAL FOR SPECIAL SURGERY,xlsx,https://www.hss.edu/files/HSS-Average-Inpatien...,"[Hospital, For, Special, Surgery]"
9,330397,INTERFAITH MEDICAL CENTER,csv,http://www.interfaithmedical.com/patients-and-...,"[Interfaith, Medical, Center]"


In [13]:
l = len(nycHospitals)

In [14]:
destFol = os.getcwd()+'/dataFiles/rawHospitalChargeData/'

for i in range(0, l):
    s=''
    url = nycHospitals['url'][i]
    name = nycHospitals['tempName'][i][0].lower()+s.join(nycHospitals['tempName'][i][1:])
    
    if nycHospitals['format'][i] == 'csv':
        try:
            urllib.request.urlretrieve(url, name+nycHospitals['providerId'][i]+'.csv')
            os.system('mv ' + name+nycHospitals['providerId'][i]+'.csv ' + destFol)
            print(i, name)
        except:
            print('An error occurred in :', name)
    elif nycHospitals['format'][i] == 'xlsx':
        try:
            resp = requests.get(url)
            output = open(name+nycHospitals['providerId'][i]+'.xlsx', 'wb')
            output.write(resp.content)
            output.close()
            os.system('mv ' + name+nycHospitals['providerId'][i]+'.xlsx ' + destFol)
            print(i, name)
        except:
            print('An error occurred in :', name)
            !curl url > name+nycHospitals['providerId'][i]+'.xlsx'
            !mv name+nycHospitals['providerId'][i]+'.xlsx' destFol
    elif nycHospitals['format'][i] == 'xls':
        try:
            resp = requests.get(url)
            output = open(name+nycHospitals['providerId'][i]+'.xls', 'wb')
            output.write(resp.content)
            output.close()
            os.system('mv ' + name+nycHospitals['providerId'][i]+'.xls ' + destFol)
            print(i, name)
        except:
            print('An error occurred in :', name)
            !curl url > name+nycHospitals['providerId'][i]+'.xls'
            !mv name+nycHospitals['providerId'][i]+'.xlsx' destFol
    else:
        print(i, name, "NA")

0 bellevueHospitalCenter
1 bronxLebanonHospitalCenter
2 brookdaleHospitalMedicalCenter
3 brooklynHospitalCenterAtDowntownCampus
4 coneyIslandHospital
5 elmhurstHospitalCenter
6 flushingHospitalMedicalCenter
7 harlemHospitalCenter
8 hospitalForSpecialSurgery
9 interfaithMedicalCenter
10 jacobiMedicalCenter
11 jamaicaHospitalMedicalCenter
12 kingsCountyHospitalCenter
13 kingsbrookJewishMedicalCenter
14 lenoxHillHospital
15 lincolnMedicalMentalHealthCenter
16 longIslandJewishMedicalCenter
17 maimonidesMedicalCenter
18 metropolitanHospitalCenter
19 montefioreMedicalCenter
20 mountSinaiBethIsrael
21 mountSinaiHospital
22 mountSinaiWest
23 nYEyeAndEarInfirmary
24 newYorkCommunityHospitalOfBrooklynInc
25 newYorkPresbyterianBrooklynMethodistHospital
26 newYorkPresbyterianHospital
27 newYorkPresbyterianQueens
28 northCentralBronxHospital
29 nyuLangoneHospitals
30 queensHospitalCenter
31 richmondUniversityMedicalCenter
32 rockefellerUniversityHospital NA
33 stBarnabasHospital
34 stJohn'SEpiscopa

In [35]:
destFol

'/nfshome/yg833/nycHospitalPricing/dataFiles/rawHospitalChargeData/'

/bin/sh: 1: Syntax error: "(" unexpected


'lincoln_medical_&_mental_health_center330080.xlsx /nfshome/yg833/nycHospitalPricing/dataFiles/rawHospitalChargeData/'